In [7]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import make_interp_spline

In [8]:
flag_project = 1 # 0--regular ; 1--repartrition; 2--voronoi

In [9]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

#name_project = 'project/output_FJSJ_16-01/'               
#name_project = 'project_repartrition/repartrition_01-03/'               
#name_project = 'project_voronoi/voronoi_01-03/'               

In [10]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/


In [11]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [12]:
dir_disp_autopick = dir_project + info_basic['rdir_disp_autopick']

In [13]:
key_subworks = info_basic['key_subworks'][0:1]
#key_subworks = info_basic['key_subworks_repick']
key_subworks

['1--08-03']

### create workspace for Dispernet

In [14]:
info_basic['rdir_disp_pick'] = 'disp_pick/'
dir_disp_pick = dir_project + info_basic['rdir_disp_pick']
if not os.path.exists(dir_disp_pick):
    os.makedirs(dir_disp_pick)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

### Autopick

In [15]:
flag_f = [5,13]

In [16]:
def transfer_autopick(key_subwork):
    global dir_disp_autopick, dir_disp, flag_f
    print(key_subwork)
    ff_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_start = flag_f[0]
    f_end = flag_f[1]
    ff_this = ff_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    cc_this = cc_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    mode = np.zeros(len(ff_this))
    np.savetxt(dir_disp_pick+'ds_'+key_subwork+'curve.txt',np.c_[ff_this,cc_this,mode], fmt='%.6f  %.6f  %i')

In [17]:
for key_subwork in key_subworks:
    #print(key_subwork)
    transfer_autopick(key_subwork)

1--08-03
